# 计算文本相似度

## 读取文件

In [41]:
from vsm_similarity import read_file, cal_tf_idf, cal_doc_norm, cal_similarity, cal_similarity_mp
file_name = '199801_clear.txt'
docs, words = read_file(file_name)

这一部分耗时2秒左右。

读取完毕后，`docs`中保存着文档的预处理信息，其中每个元素都是一篇文档，以`docs[0]`为例：

```json
{
    "id": "01-01-001",
    "max_tf": 24,
    "length": 626,
    "words": ["经验/n", "永远/d", ...],
    "content": "迈向充满希望的新世纪——一九九八年新年讲话..."
} 
```

`words`则保存着所有词语在不同文档中的词频，形如：

```json
{
  "迈向/v": {
    0: 2,
    2: 1,
    36: 1,
    ...,
    2969: 1
  },
  "充满/v": {
    0: 3,
    1: 3,
    2: 2,
    ...,
    3125: 1
  },
  ...
}
```


## 预计算

In [42]:
tf_idfs = cal_tf_idf(docs, words)
cal_doc_norm(docs, tf_idfs)

这一部分耗时1秒以内。

计算所有词语在不同文档中的tf-idf，并保存在相应的数据结构中，形如：
```json
{
  "迈向/v": {
    0: 0.40297056538909104,
    2: 0.4835646784669093,
    36: 0.6044558480836366,
    ...,
    2969: 0.11794260450412422
  },
  "充满/v": {
    0: 0.43890347840841126,
    ...
  },
  ...
}
```

然后由得到的tf-idf计算各个文档向量的模长，保存在`docs`的新字段中。

## 计算相似度

In [43]:
sim_mp = cal_similarity_mp(docs, tf_idfs, pnum=4)

In [44]:
sim = cal_similarity(docs, tf_idfs)

这里提供了两种计算方式，单进程和多进程。其中单进程耗时35秒以上，并行数4的多进程耗时25秒以内，具体时间可能有所偏差，视运行环境而定。

In [45]:
import math
all([math.isclose(a, b) for a, b in zip(sim, sim_mp)])

True

可以验证两种计算方式得到的结果是一致的。

## 验证结果

In [46]:
def get_simililarity(i, j):
    """获得文档i和文档j的相似度"""
    assert len(docs) > j
    assert j > i
    return sim[j-i + (2*len(docs) - i - 1)*i//2 - 1]


def get_specific(i):
    """获得特定文档i与其他文档的相似度"""
    assert len(docs) > i
    ret = []
    for j in range(i):
        ret.append(get_simililarity(j, i))
    ret.append(1)
    index = ((2*len(docs) - i - 1)*i)//2
    ret += sim[index:index+len(docs)-i-1]
    return ret


def most_similar(i, n=20):
    """查找与文档i最相似的文档"""
    v = get_specific(i)
    return sorted(((j, v[j]) for j in range(len(docs))), key=lambda x: x[1], reverse=True)[:n]

这里定义了一些实用函数，方便后续的结果验证，可自行任意调用。

In [57]:
import random

target = random.randint(0, len(docs)-1)
res = most_similar(target)
print(docs[target]['content'])      # 目标文档的内容
print()                             # 换个行
print(docs[res[1][0]]['content'])   # 最相似的文档的内容
print(res)                          # 具体相似度值

迎接更辉煌的新世纪
中国文联副主席李默然
近一个世纪以来，话剧始终和中国人民反帝、反封建、创建新中国、建设社会主义一系列伟大斗争与变革同行。因此，它赢得了广大观众的认同、爱护。
对已仙逝的诸多先行者与千万名今天犹在话剧战线上拼搏、战斗的『话剧人』来说，取得的重要经验之一，就是千方百计争取广大观众到剧场来。因为有没有观众，不仅是个票房价值问题。话剧不同于其他艺术门类的一个独有的艺术规律，就是一出话剧的最后完善、完成，必须有广大观众的参与和创造。所谓千百次不同演出，有千百次新的体验与体现，正是从广大观众的反馈中得来的。反映现实生活的剧目也好，上演古今中外的经典名著也好，皆同此理。
因此，无论进行什么样的探索、实验，我们的聚焦点，始终应是广大观众所思、所想以及他们的欣赏习惯与美学趣味，要努力争取广大观众的支持。因为没有观众，就没有话剧，这是个常识性的问题。什么时候离开了这个重要的宝贵经验，就势必遭到广大观众的冷漠，话剧也势必陷入困境，实践已证明了这一点。
党的十五大向全党、全国各族人民提出宏伟的蓝图。江泽民同志在报告中提出建设有中国特色社会主义文化的重大任务。他说：『建设有中国特色的社会主义文化，是凝聚和激励各族人民的重要动力，是综合国力的重要标志。』并明确指示，艺术的根是中国五千年文化，源是沸腾的现实生活。他指出，借鉴外国先进文化，要为我所用。这就从文化工作的宗旨、目的、方法，都作了明确的、科学的阐述。应该说我们是生逢盛世，方向明确，任务光荣。话剧工作者一定要明确重任，努力贯彻。
江泽民同志最近指出：『话剧有它特殊的魅力，是别的艺术形式不能代替的。』话剧工作者因此受到极大鼓舞。相信总书记对话剧艺术的评价，会增强话剧队伍的凝聚力与信心。
话剧工作的九十年历程，有经验、有教训，有辉煌亦有黯淡。但是，只要我们不脱离群众，不忽视广大观众的正当需求，又十分认真地遵守话剧本身的艺术规律，话剧不仅不会消亡，还会有新的创造，有更辉煌的前程。
祝愿我们的话剧事业更加辉煌！


话剧再次成为热门话题艺术家、评论家在京聚会讨论繁荣话剧艺术
本报北京１月８日讯新年伊始，中国话剧９０年纪念活动，不仅使首都文艺舞台更加多姿多彩，也使“话剧”再次成为广大观众和各界人士关注的话题。今天，我国部分知名话剧艺术家、戏剧评论家聚集一起，就如何进一步繁荣话剧问题各抒己见。
艺术家们对江泽民总书记日前观看话剧《